# Data Processing Procedure

This notebook shows how to use the provided Python code that defines a DataProcessor class designed to process WiFi Information from the campus of the University of Colorado Boulder. This class deals with multiple Wi-Fi datasets, each corresponding to a different building on the campus. The input data are stored in CSV files, and the class processes these files for future extraction of useful insights.

Date Created: July 22, 2023  
By Tyler Reiser

---

## Data Overview: Campus WiFi

### Input

The Wifi Information for each building on the University of Colorado Boulder campus stored as `CSV` files.

### Data Format

The Wifi Information for each building on campus from \[2019-08-16 08:10:53\] to \[2021-05-25 06:57:08\]. The data is segmented into three folders: “UCB Wireless”, “UCB Guest”,  and “Eduroam”; each Folder contains data sets from each network . Every file is a CSV file named in the following format: \[Building Code\]_Extracted_Data_\[Start Date\]. 

Each `.csv` file contains the date and time in column zero and the active connected device count in column one. The devices recorded are connected to access points in building rooms, where most rooms have one access point and some have more than one. 

An example of file data is given below from the UCB Wireless network in Aden Hall. This first row of this example dataset would be read as “In Aden Hall, on August 16, 2019, at 8:10:53, there were eight devices connected to UCB Wireless”.

### EXAMPLE: ADEN HALL

| datetime |   devicecount  | 
| -------- |    --------    |
| 2019-08-16 08:10:53 | 8.0 |
| 2019-08-16 08:18:58 | 4.0 |
| 2019-08-16 08:28:17 | 5.0 |
| 2019-08-16 08:32:10 | 9.0 |
| 2019-08-16 08:37:38 | 6.0 |
| 2019-08-16 08:42:51 | 9.0 |
| 2019-08-16 08:47:59 | 5.0 |

### Setting Directory Paths

Store `.csv` files at `BASE_DIR/data/input/WiFiData`.

### Output

The result of the `process_all_buildings` method is a list of buildings to process (either all buildings or a specific building if `building_id` is provided), and uses the joblib's `Parallel` function to process the data for these buildings in parallel, making use of multiple CPU cores if available. The number of cores used for parallel processing is determined by the `cpu_cores` parameter passed during the initialization of the `DataProcessor` object. The method returns a dictionary where the keys are building names and the values are the processed DataFrames for each building. This design enhances performance, especially when processing large amounts of data across multiple files.

---

## Classes in this Module

The `DataProcessor` class focuses on data processing, while the `SpeedTest` class provides a reusable way to measure execution time. This is an example of object-oriented design.


### Overview

The `DataProcessor` class is responsible for processing time-series data from CSV files. It reads the data, interpolates missing values, and processes data for all buildings in parallel. The class is designed to be flexible, allowing the user to specify parameters like the data directory, date range, building ID, CPU cores, and sampling frequency.

The `SpeedTest` class is a utility class that measures the execution time of a function or method. It's implemented as a callable class, which means it can be used as a decorator to easily add timing functionality to any function or method. In this case, it's used to measure the time taken by the process_all_buildings method of the DataProcessor class.

### The `DataProcessor `Class

1. Initialization Method (`__init__`):

The `DataProcessor` class is initialized with parameters which include the directory where the CSV files are stored, the start and end dates for the data to be considered, a specific building_id if the user would like to process data for a particular building only, the number of CPU cores to utilize for the computation, the frequency at which to sample data, and a boolean to decide whether to record the total execution time.

The `DataProcessor` class used the following methods internally:

2. Reading Data Method (`read_time_series_data`):

The `read_time_series_data` method is a static method, meaning it does not depend on the state of the DataProcessor instance. This method reads data from a CSV file and returns a DataFrame. It also converts the "datetime" column to actual datetime objects for easy handling in subsequent methods.

3. Interpolating Data Method (`interpolate_time_series_data`):

The `interpolate_time_series_data` method implements interpolation on the provided data. This is useful to fill gaps in the dataset where data points might be missing. With the help of the pandas.date_range function, a date range object is created from the minimum date to the maximum date with a frequency set by the `sample_freq` attribute. The method then interpolates the missing data points and returns an updated DataFrame. The method is shown below:

4. Building Identifiers Method (`_get_building_identifiers` and `_get_buildings`):

The `_get_building_identifiers` is a static method that retrieves the building identifiers from a given filename. It splits the filename, which has the structure `buildingID_Extracted_Data_date.csv`, into a list using "_" as the delimiter and then returns the building id.

5. Data Processing Method (`_process_building_data` and `process_all_buildings`):

The `_process_building_data` method processes the data for a single building. It uses the `read_time_series_data` method to read the dataset and `interpolate_time_series_data` to interpolate missing data.

The `process_all_buildings` method applies the above steps to all buildings. It first gets the list of buildings to process, either a single one if `building_id` is provided, or all buildings in the directory otherwise. Then, using joblib's Parallel and delayed functions, it processes data for all these buildings in parallel, making efficient use of multiple CPU cores if available. It returns a dictionary where keys are building names and values are DataFrames containing corresponding processed data.

If `record_time` is set True, it records and returns execution time for data processing. This is used for performance testing.

### The `SpeedTest` Class

This is a wrapper for the DataProcessor class and is used to measure the execution time of any function or method. In this package, it is only being used to record the time taken to process the data.

---


1. Import the class

In [1]:
from src.python import DataProcessor as dp

2. Create an instance of the `DataProcessor` class using the the `process_all_buildings` method

In [2]:
data_processor  = dp.DataProcessor()
data            = data_processor.process_all_buildings()

3. Call both columns of data for the Aerospace building like this:

In [3]:
data[   'AERO'  ].head(5)

,datetime,devicecount
0,2019-08-16 08:10:53,92.000000
1,2019-08-16 08:12:53,93.237113
2,2019-08-16 08:14:53,94.474227
3,2019-08-16 08:16:53,95.711340
4,2019-08-16 08:18:53,96.948454


In [4]:
data[   'ATLS'  ][  'devicecount'   ].head(5)

0    37.0
1    37.0
2    37.0
3    37.0
4    37.0
Name: devicecount, dtype: float64

4. Call the first column of data for an individual building:

### Optional Parameters

The simplest use of the `DataProcessor` class is to pass no values, which is shown above. The following section demonstrates options for improving performance of the processing procedure and initialization of several parameters.

---
**Parameter List**

1. `csv_directory`: used to specify the CSV files directory.
2. `start_date` and `end_date`: the start and end dates for data processing.
3. `building_id`: a building ID for processing data related to a specific building.
4. `cpu_cores`: the number of CPU cores to use for parallel processing.
5. `sample_freq`: the time between each interpolated point.
6. `record_time=True`: performance testing only.

---


1. Set `csv_directory` based on the path to the directory with the CSV files.

In [5]:
data_processor = dp.DataProcessor( csv_directory   =   './data/input/WiFiData/'    )
data           = data_processor.process_all_buildings()

data['AERO'].head(5)

,datetime,devicecount
0,2019-08-16 08:10:53,92.000000
1,2019-08-16 08:12:53,93.237113
2,2019-08-16 08:14:53,94.474227
3,2019-08-16 08:16:53,95.711340
4,2019-08-16 08:18:53,96.948454


2. Set `cpu_cores` based on the available cores on your machine for optimal performance.

In [6]:
data_processor = dp.DataProcessor( cpu_cores   =   10  )
data           = data_processor.process_all_buildings()

data['ADEN'].head(5)

,datetime,devicecount
0,2019-08-16 08:10:53,8.000000
1,2019-08-16 08:12:53,7.010309
2,2019-08-16 08:14:53,6.020619
3,2019-08-16 08:16:53,5.030928
4,2019-08-16 08:18:53,4.041237


3. Set `sample_freq` if you want to interpolate between points at an interval that is different than `10Min`.

In [7]:
data_processor = dp.DataProcessor( sample_freq =   '20Min'  )
data_frequency = data_processor.process_all_buildings()

data_frequency['ATLS'].head(5)

,datetime,devicecount
0,2019-08-16 08:10:53,37.000000
1,2019-08-16 08:30:53,38.339056
2,2019-08-16 08:50:53,39.410811
3,2019-08-16 09:10:53,47.601286
4,2019-08-16 09:30:53,51.946372


4. Set a `start_date` and/or `end_date`.

In [8]:
from datetime import datetime


data_processor  = dp.DataProcessor( start_date  =   datetime( 2019, 10, 1  ), 
                                    end_date    =   datetime( 2020,  4, 1  )    
                                    )

data_range      = data_processor.process_all_buildings()

data_range['AERO'].head(5)

,datetime,devicecount
0,2019-10-01 00:00:00,89.000000
1,2019-10-01 00:02:00,89.000000
2,2019-10-01 00:04:00,89.000000
3,2019-10-01 00:06:00,88.680995
4,2019-10-01 00:08:00,87.866516


5. Set `building_id` to study a specific building in the form `ATLS`

In [9]:
data_processor  = dp.DataProcessor ( building_id    =   'ATLS')
data_atlas      = data_processor.process_all_buildings()

6. The `record_time` parameter is for performance testing using in the next section.


---
## Speed-Up

To illustrate the performance gained by implementing this parallel processing stratigy, a loop was implemented to run the `DataProcessor` for each core, ranging from 1 to the maximum number of CPU cores. This was done for interpolation between points with each of the following sample frequencies: `['15Min','10Min','5Min','2Min']` and `['6Min','5Min','4Min']`. **The total time taken for each run was recorded and plotted, providing a visual representation of the performance increase when using multiple CPU cores.** The time was recored into a Python dictionary and converted to a Pandas DataFrame before being returned by the function. 

In [10]:
from src.python import DataProcessor as dp

processor   = dp.DataProcessor(record_time=True)
processor.process_all_buildings()

datetime.timedelta(seconds=1, microseconds=145005)

### Result

The processing of data interpolation for 144 points across 77 buildings was significantly expedited by utilizing all 10 cores on a Apple M1 Pro with a 10-core processor and 32 gb of unified memory. The time required for this operation was reduced from 5.2 seconds to 1.3 seconds, demonstrating the efficiency of parallel processing independent CSV files, but it isn't a massive jump in performance The output data remained consistent, despite the increased processing speed, and was not printed again for brevity. This improvement in performance uses a straightforward strategy where each processing unit operates on a `.csv` file independently, as the memory does not depend on each other, By default, the system will utilize the maximum number of CPU cores available. However, if any issues arise, one can manually set `cpu_cores = 1` to only execute one process at a time.

### Future Work

This notebook presents a speed-up for the processing procedure on my laptop when it is not connected to a power source. Running this on an HPC cluster would increase the speed exponentially. 